GPU確認

In [ ]:
!nvidia-smi

driveマウント

In [ ]:
from google.colab import drive
drive.mount("./googledrive")

Mounted at ./googledrive


初回のみ:GoogleDriveにmodelとVAEの保存ディレクトリを作成

In [ ]:
%cd /content/googledrive/MyDrive/
!mkdir model vae

model,VAE等のダウンロード

In [ ]:
download = 'vae' #@param ["model", "vae"] {allow-input: true}

In [ ]:
path=''
if download=='model':
  path="/content/googledrive/MyDrive/model"
elif download=='vae':
  path="/content/googledrive/MyDrive/vae"
else:
  print("ダウンロードできません")

vae


In [ ]:
%cd $path

/content/googledrive/MyDrive/model


huggingface以外からのダウンロード

In [ ]:
url="https://civitai.com/api/download/models/19053"#@param{type:"string"}
!wget $url

名前変更

In [ ]:
filename=""#@param{type:"string"}
rename=""#@param{type:"string"}
!mv $filename $rename

huggingface内のmodel、VAEのダウンロード

In [ ]:
!pip install --upgrade huggingface_hub

ダウンロードに失敗したら2回目以降はこのセルのみ実行
（ダウンロードできないものもある）

In [ ]:
from huggingface_hub import hf_hub_download
model_id="WarriorMama777/OrangeMixs"#@param{type:"string"}
file_name="orangemix.vae.pt"#@param{type:"string"}
path=hf_hub_download(repo_id=model_id, filename=file_name,revision="main")

webuiのclone

In [ ]:
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
%cd stable-diffusion-webui

使うmodel,vaeの選択


guiライブラリのダウンロード

In [ ]:
! pip install -U ipywidgets

model or vaeの選択

In [ ]:
from ipywidgets import widgets
from IPython.display import display
import glob
import os
download = 'model' #@param ["model", "vae"] {allow-input: true}
path=''
if download=='model':
  path="/content/googledrive/MyDrive/model/*.*"
elif download=='vae':
  path="/content/googledrive/MyDrive/vae/*.*"
else:
  print("modelかvaeを選択してください。")

filelist=glob.glob(path)
filename=[os.path.basename(f) for f in filelist]

w = widgets.RadioButtons(
    options=filename,
    description="choose file",
    disabled=False
)

display(w)

RadioButtons(description='choose file', options=('Defmix-v2.0.safetensors',), value='Defmix-v2.0.safetensors')



VAE or model の設定(MyDrive/vae/以下に置く)

In [ ]:
model_path="/content/googledrive/MyDrive/"+download+'/'+w.value
!cp $model_path /content/stable-diffusion-webui/models/Stable-diffusion/

extensionの読み込み

In [ ]:
import os
github_url=""#@param{type:"string"}
ex_filename=os.path.basename(github_url)
path="/content/stable-diffusion-webui/extensions/"+ex_filename
!git clone $github_url $path

webui設定の読み込み

In [ ]:
!cp /content/googledrive/MyDrive/config.json /content/stable-diffusion-webui/config.json

webuiの実行

In [ ]:
!pip install xformers=0.0.18

In [ ]:
%cd /content/stable-diffusion-webui
#起動せずインストールのみ行う
!python launch.py --exit

In [ ]:
!COMMANDLINE_ARGS="--share --gradio-debug --opt-sdp-attention　--xformers --no-half-vae" REQS_FILE="requirements.txt" python launch.py

webuiの変更した設定を保存

In [ ]:
!cp /content/stable-diffusion-webui/config.json /content/googledrive/MyDrive/config.json

uiモジュールのインストール

In [ ]:
!pip install -U gradio

プロンプト保存機能の立ち上げ、
ディレクトリ移動/prompt(改良予定)

In [ ]:
!mkdir -p /content/googledrive/MyDrive/prompt

/content/googledrive/MyDrive
mkdir: cannot create directory ‘prompt’: File exists


In [ ]:
%cd /content/googledrive/MyDrive/prompt

プロンプトの保存機能

In [ ]:
import gradio as gr
import os
import glob

class File_list:

  def __init__(self):
    self.file_list=[]

  def __call__(self):
    self.file_list=[]
    files=glob.glob("/content/googledrive/MyDrive/prompt/*")
    for f in files:
      self.file_list.append(os.path.split(f)[1])
    return self.file_list

def save(file_name,prompt):
  path="/content/googledrive/MyDrive/prompt/{}.txt".format(file_name)
  with open(path,'x') as f:
    try:
      f.write(prompt)
      return "save completed"
    except Exception as e:
      return ("cant save")  

def load(file_name):
  path="/content/googledrive/MyDrive/prompt/{}".format(file_name)
  with open(path,'r') as f:
    try:
      return f.read()
    except Exception as e:
      return"cannot find a file"

def delete(file_name):
  path="/content/googledrive/MyDrive/prompt/{}".format(file_name)
  os.remove(path)
  file_list()
  return "delete"

file_list=File_list()
with gr.Blocks() as spmt:
  gr.Markdown("save prompt and load prompt")
  with gr.Tabs():
    with gr.TabItem("save prompt"):
      prompt_input=gr.Textbox(label="prompt:")
      file_name1=gr.Textbox(label="filename")
      text_output1=gr.Textbox(label="save")
      text_button1=gr.Button("save prompt")
    with gr.TabItem("load prompt"):
      file_name2=gr.Dropdown(file_list(),label="load prompt")
      prompt_output=gr.Textbox(label="load")
      text_button2=gr.Button("load prompt")
    with gr.TabItem("Delete prompt"):
      file_name3=gr.Dropdown(file_list(),label="delete prompt")
      d_out=gr.Textbox(label="delete")
      d_button=gr.Button("Delete")

  text_button1.click(save,inputs=[prompt_input,file_name1],outputs=text_output1)
  text_button2.click(load,inputs=file_name2,outputs=prompt_output)
  d_button.click(delete,inputs=file_name3,outputs=d_out)

spmt.launch()

ダウンロードしたmodelをdriveから削除（改良予定）

In [ ]:
import os
agree=True #@param{type:"boolean"}
remove_filename="convert_pt_to_bin.py" #@param{type:"string"}
path="/content/googledrive/MyDrive/model/"+remove_filename
if True==True:
  os.remove(path)
else:
  print("Click check box and Tri again")

ダウンロードしたvaeをdriveから削除（改良予定）

In [ ]:
import os
agree=True #@param{type:"boolean"}
remove_filename="convert_pt_to_bin.py" #@param{type:"string"}
path="/content/googledrive/MyDrive/vae/"+remove_filename
if True==True:
  os.remove(path)
else:
  print("Click check box and Tri again")


生成した画像の閲覧と保存したい画像をドライブに移動(実装予定)